In [1]:
import logging

import probtorch
import torch

import combinators
import hmm
import importance
import mcmc
import trace_tries
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [3]:
init_hmm = combinators.HyperPopulation(hmm.init_hmm, (1,), hyper=hmm_params)
hmm_step = combinators.PrimitiveCall(hmm.hmm_step)
hmm_run = combinators.Reduce(hmm_step, generator=lambda: range(50), initializer=init_hmm)

In [4]:
(z_last, mu, sigma, pi, pi0), generative = hmm_run()

In [5]:
data = {k.rpartition('/')[-1]: rv.value for (k, rv) in generative.filter(lambda k, rv: 'X_' in k)}

In [6]:
num_particles = 250

In [7]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    smc_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [8]:
init_hmm = combinators.HyperPopulation(hmm.init_hmm, (num_particles,), trainable=smc_hmm_params)

In [9]:
smc_hmm = importance.smc(hmm_step, (num_particles,), lambda: range(50), initializer=init_hmm)

In [10]:
particle_mh = mcmc.IndependentMH(smc_hmm, num_particles*10)

In [ ]:
samples, inference = particle_mh(trace=trace_tries.HierarchicalTrace(observations=utils.dict_lookup(data)))

In [ ]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inference[key].value == generative[key].value).sum().to(dtype=torch.float) / inference.num_particles
    print('SMC accuracy at time %d: %f' % (t, accuracy))